In [1]:
from PIL import Image
import numpy as np
from collections import Counter
import colorsys

# Define rainbow colors in RGB
RAINBOW_COLORS = {
    'Red': (255, 0, 0),
    'Orange': (255, 165, 0),
    'Yellow': (255, 255, 0),
    'Green': (0, 255, 0),
    'Blue': (0, 0, 255),
    'Indigo': (75, 0, 130),
    'Violet': (238, 130, 238)
}

def calculate_average_brightness(image):
    grayscale_image = image.convert('L')  # Convert to grayscale
    pixels = np.array(grayscale_image)
    return np.mean(pixels)  # Return average brightness

def calculate_saturation(image):
    pixels = np.array(image)
    hsv_pixels = [colorsys.rgb_to_hsv(r/255.0, g/255.0, b/255.0) for r, g, b in pixels.reshape(-1, 3)]
    saturations = [s for h, s, v in hsv_pixels]
    return np.mean(saturations)  # Return average saturation

def find_rainbow_colors(image, rainbow_colors):
    # Convert image to RGB
    pixels = np.array(image.convert('RGB'))
    
    # Flatten the image array and create a color Counter
    flattened_pixels = pixels.reshape(-1, 3)
    pixel_counter = Counter(map(tuple, flattened_pixels))

    # Compare each pixel with rainbow colors
    rainbow_frequency = {color: 0 for color in rainbow_colors.keys()}
    for pixel, count in pixel_counter.items():
        for color_name, color_value in rainbow_colors.items():
            if np.linalg.norm(np.array(pixel) - np.array(color_value)) < 50:  # Tolerance for color matching
                rainbow_frequency[color_name] += count
    
    # Order colors by frequency
    ordered_colors = sorted(rainbow_frequency.items(), key=lambda x: x[1], reverse=True)
    return ordered_colors

def assign_music_features(brightness, ordered_colors, saturation):
    # Assign pitch based on brightness (A4 = 440Hz is middle range)
    base_pitch = 440  # Middle A pitch
    pitch = int(base_pitch * (brightness / 255))  # Scale brightness to pitch range
    
    # Assign sequence of tones based on color order (map each color to a musical note)
    color_to_note = {
        'Red': 'C',
        'Orange': 'D',
        'Yellow': 'E',
        'Green': 'F',
        'Blue': 'G',
        'Indigo': 'A',
        'Violet': 'B'
    }
    tone_sequence = [color_to_note[color] for color, freq in ordered_colors if freq > 0]

    # Assign tempo based on saturation (higher saturation, faster tempo)
    base_tempo = 120  # Base tempo in BPM
    tempo = int(base_tempo * saturation)  # Scale saturation to tempo
    
    return pitch, tone_sequence, tempo

def process_image(image_path):
    image = Image.open(image_path)
    
    # Extract features
    brightness = calculate_average_brightness(image)
    saturation = calculate_saturation(image)
    ordered_colors = find_rainbow_colors(image, RAINBOW_COLORS)
    
    # Assign music features
    pitch, tone_sequence, tempo = assign_music_features(brightness, ordered_colors, saturation)
    
    # Print the results
    print(f"Average Brightness: {brightness}")
    print(f"Color Frequencies: {ordered_colors}")
    print(f"Average Saturation: {saturation}")
    print(f"Assigned Pitch (Hz): {pitch}")
    print(f"Tone Sequence: {tone_sequence}")
    print(f"Assigned Tempo (BPM): {tempo}")
    
    return {
        "brightness": brightness,
        "ordered_colors": ordered_colors,
        "saturation": saturation,
        "pitch": pitch,
        "tone_sequence": tone_sequence,
        "tempo": tempo
    }

# Example usage
image_path = '001_pillars_of_creation_new.jpg'  # Replace with your image path
process_image(image_path)


FileNotFoundError: [Errno 2] No such file or directory: '/home/gustamatos/Documents/spaceapps2024/StarMonics/001_pillars_of_creation_new.jpg'

In [4]:
from openai import OpenAI
from openai import AssistantEventHandler
from typing_extensions import override

# Initialize the OpenAI API client
client = OpenAI(api_key='sk-FDti6mHcD9tIT6bNgmrHvPd9bUcUqCJRfJLhynB9OkT3BlbkFJSvdRzIA5BrDL_GKIhBkFTKl-fZKmmSYoUMizf8ZEMA')

# Create an assistant for song generation
assistant = client.beta.assistants.create(
    name="Song Composer",
    instructions="You are a music assistant capable of generating Hans Zimmer inspired songs using ABC notation. The song should evoke a sense of cosmic wonder, with low-pitch, symphonic instrumentation. Only output the ABC notation, including instruments used.",
    tools=[{"type": "code_interpreter"}],  # Assuming Song Maker is a tool you can integrate
    model="gpt-4o"  # Specify the model
)

# Create a conversation thread
thread = client.beta.threads.create()

# Send the initial message (prompt)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
    Create a grandiose orchestral song inspired by the following prompt:
    'In the cosmic expanse, a nebula blooms,
    Swirls of gas and dust like a dream in hues.
    Golden cliffs rise, sculpted by stellar winds,
    Reaching into the void where eternity begins.'

    Style: Symphonic, in the style of Hans Zimmer.
    Tone Sequence: A - B - C# - G#
    Tempo: 91 BPM
    Pitch: Low-pitch, inspirational and exploratory.
    Output the song only in ABC notation with its respective instruments.
    Do not output nothing more than the song in ABC notation and its respective instrumets. Remove it if there is anything else.
    """
)

# Define an event handler to handle the response stream
class SongMakerEventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > {text}", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        if delta.value:
            print(delta.value, flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > Song generation in progress using {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'song_maker' and delta.song_maker.outputs:
            for output in delta.song_maker.outputs:
                if output.type == "abc_notation":
                    print(f"ABC Notation:\n{output.abc_notation}", flush=True)

# Stream the response using the event handler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=SongMakerEventHandler(),
) as stream:
    stream.until_done()



assistant > Text(annotations=[], value='```')
```
abc


X
:
1


T
:
Cos
mic
 Bloom


M
:
4
/
4


L
:
1
/
8


Q
:
1
/
4
=
91


K
:
Am



%%
M
IDI
 instrument
 
48
 
 %
 String
 Ensemble
 
1


V
:
1
 name
="
Strings
"
 

[A
,,
2
E
,,
2
]
 |
 [
A
,,
C
,,
G
,,
]
 [
A
,,
C
,,
G
,,
]
 [
C
,,
E
,,
A
,,
]
 z
2
 |
 [
E
,,
G
,,
C
,,
]
 [
E
,,
G
,,
C
,,
]
 [
G
,,
B
,,
E
,,
]
 z
2
 |
 

[A
,,
C
,,
G
,,
]
 [
A
,,
C
,,
G
,,
]
 [
C
,,
E
,,
A
,,
]
 z
2
 |
 [
E
,,
G
,,
B
,,
A
,,
]
 [
E
,,
G
,,
B
,,
A
,,
]
 [
G
,,
B
,,
C
,,
]
 z
2
 ||


%%
M
IDI
 instrument
 
42
 
 %
 C
ello


V
:
2
 name
="
Cell
o
"

C
,,
4
 |
 C
,,
4
 |
 G
,,
4
 |
 G
,,
4
 |

C
,,
4
 |
 C
,,
4
 |
 E
,,
4
 |
 E
,,
4
 ||


%%
M
IDI
 instrument
 
68
 
 %
 French
 Horn


V
:
3
 name
="
French
 Horn
"

A
,,
2
 B
,,
2
 |
 C
,,
2
 G
,,
2
 |
 A
,,
2
 C
,,
2
 |
 E
,,
2
 A
,,
2
 |

B
,,
2
 C
,,
2
 |
 G
,,
2
 B
,,
2
 |
 E
,,
2
 C
,,
2
 |
 G
,,
2
 A
,,
2
 ||


%%
M
IDI
 instrument
 
62
 
 %
 Brass
 Section


V
:
4
 name
="
Br
ass
"

[A
,,,
2
E
,

In [11]:
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler

# Initialize the OpenAI API client
client = OpenAI(api_key='sk-FDti6mHcD9tIT6bNgmrHvPd9bUcUqCJRfJLhynB9OkT3BlbkFJSvdRzIA5BrDL_GKIhBkFTKl-fZKmmSYoUMizf8ZEMA')

# Create an assistant for song generation
assistant = client.beta.assistants.create(
    name="Song Composer",
    instructions="You are a music assistant capable of generating Hans Zimmer inspired songs using ABC notation. The song should evoke a sense of cosmic wonder, with low-pitch, symphonic instrumentation. Only output the ABC notation, including instruments used.",
    tools=[],  # Assuming Song Maker is a tool you can integrate
    model="gpt-4o"  # Specify the model
)

# Create a conversation thread
thread = client.beta.threads.create()

# Send the initial message (prompt)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
    Hello! Write and export a song that embodies the following prompt:

"In the cosmic expanse, a nebula blooms,
Swirls of gas and dust like a dream in hues.
Golden cliffs rise, sculpted by stellar winds,
Reaching into the void where eternity begins.

Celestial clouds curl, a dance of light and shade,
Where young stars are born, their brilliance displayed.
A sea of sapphire and amber intertwined,
In this ethereal realm where time is confined."

Aim for a polished, well thought out song. On a style close to John Williams. Create a grandiose-feeling, orchestra-sounding symphonic arrangement. It should feel very inspirational and it should feel like the poem above. Like you want to explore the vastness of space. Try to keep it well-organized. Low pitch. 

Tone Sequence: A - B - C# - G#
Assigned Tempo (BPM): 91
Pitch: Low pitch all throughout.
First, write the introduction.
Write it in ABC notation.
    """
)

# Define an event handler to handle the response stream
class SongMakerEventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > {text}", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        if delta.value:
            print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > Song generation in progress using {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'song_maker':
            if delta.song_maker.input:
                print(f"SongMaker input: {delta.song_maker.input}", flush=True)
            if delta.song_maker.outputs:
                print(f"\n\nSong generated successfully!\n", flush=True)
                for output in delta.song_maker.outputs:
                    if output.type == "midi":
                        print(f"MIDI file URL: {output.midi_url}", flush=True)
                    if output.type == "abc_notation":
                        print(f"ABC Notation: {output.abc_notation}", flush=True)

# Stream the response using the event handler
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=SongMakerEventHandler(),
) as stream:
    stream.until_done()


assistant > Text(annotations=[], value='```')```abc
X:1
T:Cosmic Wonders: Overture
C:Inspired by Hans Zimmer
L:1/8
Q:1/4=91
M:4/4
I: [V1] "Piano" [I:pc -3] 
I: [V2] "Strings" [I:pc -2]
I: [V3] "Woodwinds" [I:pc -4]
I: [V4] "Brass" [I:pc -1]
K:Emin

V:1
|: P:Intro
V:1 in "Piano" z4 [EA,]2- | [EA,]2 [G,B,]2 [F,B,]2 z2 | [E,2A,2] z [EB,]3- | [EB,]2 [D,B,2] z2 [A,2E,2] :|

V:2
|: P:Intro
V:2 in "Strings" [E,,A,,]4- | [E,,A,,][G,,B,,][F,,B,,] [E,,A,,] z2 | [E,,A,,]2 [E,,B,,]2 [D,,B,,] z4 | [A,,E,,]2 [G,,D,,]2 z2 :|

V:3
|: P:Intro
V:3 in "Woodwinds" z4 [E2A2][G2B2][D3F3]| [E2A2] [G2B2][D2F2] z2| [E2A2] z [G2B2]| [D2F2)[D2B2] z2[A2E2]z2:|

V:4
|: P:Intro
V:4 in "Brass" z2 [E,,A,,]4 | [E,,A,,]4 [G,,B,,]3 | [D,,B,,]4 [F,,B,,]2 z | [A,,E,,]4 z2 [E4] :|

```


In [59]:
import requests
import json
import os

api_key = 'sk-FDti6mHcD9tIT6bNgmrHvPd9bUcUqCJRfJLhynB9OkT3BlbkFJSvdRzIA5BrDL_GKIhBkFTKl-fZKmmSYoUMizf8ZEMA'
input_text = 'In the cosmic expanse, a nebula blooms, Swirls of gas and dust like a dream in hues. Golden cliffs rise, sculpted by stellar winds, Reaching into the void where eternity begins.'

class SongMaker:
    def __init__(self, api_key, input_text):
        self.api_key = api_key
        self.input_text = input_text
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }


    def make_song(self):
        """Makes an ABC format song based on a input text."""
        payload = {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "user",
                    "content": "RolePlay as a musical bot. Generate a a music in ABC format based on the input text."
                },
                {
                    "role": "user",
                    "content": f"Style: Symphonic, in the style of John Williams. Tone Sequence: A - B - C# - G#. Tempo: 91 BPM. Pitch: Low-pitch, inspirational and exploratory. {self.input_text}"
                },
                {
                    "role": "user",
                    "content": "Output the correspondent instruments."
                },
                {
                    "role": "user",
                    "content": "Remove any introductions. Send only the requested text."
                },
            ],
            "max_tokens": 300
        }

        response = requests.post("https://api.openai.com/v1/chat/completions",
                                 headers=self.headers, json=payload)
        return response.json()


generate_song = SongMaker(api_key, input_text)

# Make a song
song = generate_song.make_song()
print(song)

{'id': 'chatcmpl-AFGDed7usetTiF9zHV06xoFcYpgjg', 'object': 'chat.completion', 'created': 1728200630, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```\nX: 1\nT: Nebula Bloom\nM: 4/4\nL: 1/8\nK: C\nQ: 1/4=91\n%%ment extend 5\n%%score (V1 V2 V3 V4 V5)\nV: V1 clef=treble\n| "Strings" A2 B2 | "Flute" C2 B2 | "Strings" C#2 G#2 | "Brass" G2 F2 |]\n% First phrase, exploring the nebula\n| "Strings" A2 B2 | "Flute" C2 B2 | "Strings" C#2 G#2 | "Brass" G2 C2 |]\n% Rising cliffs\nV: V2 clef=treble\n| "Woodwinds" E2 D2 | "Horns" F#2 E2 | "Woodwinds" A2 G#2 | "Horns" G2 F#2 |]\n% Echoing through the expanse\n| "Woodwinds" E2 D2 | "Horns" G#2 G2 | "Woodwinds" F#2 E2 | "Strings" A2 C2 |]\n% Reaching into the void\nV: V3 clef=bass\n| "Cello" C2 C2 | "Double Bass" B,2 A,2 | "Cello" D2', 'refusal': None}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 136, 'completion_tokens': 300, 'total_tokens': 436, 'prompt_tokens_de

In [60]:
print('\nX: 1\nT: Nebula Bloom\nM: 4/4\nL: 1/8\nK: C\nQ: 1/4=91\n%%ment extend 5\n%%score (V1 V2 V3 V4 V5)\nV: V1 clef=treble\n| "Strings" A2 B2 | "Flute" C2 B2 | "Strings" C#2 G#2 | "Brass" G2 F2 |]\n% First phrase, exploring the nebula\n| "Strings" A2 B2 | "Flute" C2 B2 | "Strings" C#2 G#2 | "Brass" G2 C2 |]\n% Rising cliffs\nV: V2 clef=treble\n| "Woodwinds" E2 D2 | "Horns" F#2 E2 | "Woodwinds" A2 G#2 | "Horns" G2 F#2 |]\n% Echoing through the expanse\n| "Woodwinds" E2 D2 | "Horns" G#2 G2 | "Woodwinds" F#2 E2 | "Strings" A2 C2 |]\n% Reaching into the void\nV: V3 clef=bass\n| "Cello" C2 C2 | "Double Bass" B,2 A,2 | "Cello" D2')


X: 1
T: Nebula Bloom
M: 4/4
L: 1/8
K: C
Q: 1/4=91
%%ment extend 5
%%score (V1 V2 V3 V4 V5)
V: V1 clef=treble
| "Strings" A2 B2 | "Flute" C2 B2 | "Strings" C#2 G#2 | "Brass" G2 F2 |]
% First phrase, exploring the nebula
| "Strings" A2 B2 | "Flute" C2 B2 | "Strings" C#2 G#2 | "Brass" G2 C2 |]
% Rising cliffs
V: V2 clef=treble
| "Woodwinds" E2 D2 | "Horns" F#2 E2 | "Woodwinds" A2 G#2 | "Horns" G2 F#2 |]
% Echoing through the expanse
| "Woodwinds" E2 D2 | "Horns" G#2 G2 | "Woodwinds" F#2 E2 | "Strings" A2 C2 |]
% Reaching into the void
V: V3 clef=bass
| "Cello" C2 C2 | "Double Bass" B,2 A,2 | "Cello" D2


In [29]:
from music21 import converter

# ABC notation for the introduction
abc_code = r"""
X:1
T:Cosmic Bloom
C:Inspired by the cosmic expanse
M:4/4
L:1/8
Q:1/4=91
K:Amin

V:1 clef=treble
V:2 clef=treble
V:3 clef=bass
V:4 clef=bass

V:1
|: A,2 C2 E2 | G2 E2 C2 | A,2 C2 E2 | G2 E2 C2 :|
|: A,2 C2 E2 | G2 E2 C2 | A,2 C2 E2 | G2 E2 C2 :|

V:2
|: A,2 B2 d2 | c2 B2 A2 | A,2 B2 d2 | c2 B2 A2 :|
|: A,2 B2 d2 | c2 B2 A2 | A,2 B2 d2 | c2 B2 A2 :|

V:3
|: A,,2 C,2 E,2 | G,2 E,2 C,2 | A,,2 C,2 E,2 | G,2 E,2 C,2 :|
|: A,,2 C,2 E,2 | G,2 E,2 C,2 | A,,2 C,2 E,2 | G,2 E,2 C,2 :|

V:4
|: A,,2 A,,2 A,,2 | G,,2 G,,2 G,,2 | A,,2 A,,2 A,,2 | G,,2 G,,2 G,,2 :|
|: A,,2 A,,2 A,,2 | G,,2 G,,2 G,,2 | A,,2 A,,2 A,,2 | G,,2 G,,2 G,,2 :|
"""

# Convert ABC to a music21 Stream
abc_score = converter.parse(abc_code, format='abc')

# Write the MIDI file
abc_score.write('midi', fp='cosmic_bloom_intro.mid')

StreamException: cannot process repeats on Stream that does not contain measures

In [44]:
# Let's attempt to convert the revised ABC notation to MIDI using a different approach

# Define the revised ABC notation
abc_code_final = """
X:1
T:Cosmic Expanse
C:Inspired by the Cosmos
M:4/4
L:1/8
Q:1/4=91
V:1 clef=treble
V:2 clef=bass
V:3 clef=treble
V:4 clef=treble

V:1
K:Amin
"A" [A,2E2] [A2F2] | "B" [B2G2] [B2E2] | "C#" [c2A2] [c2G2] | "G#" [g2e2] [g2B2] |
"A" [A,2E2] [A2F2] | "B" [B2G2] [B2E2] | "C#" [c2A2] [c2G2] | "G#" [g2e2] [g2B2] |

V:2
K:Amin
"Am" [A,2C2] [E,2C2] | "Bm" [B,2D2] [F,2D2] | "C#m" [C,2E2] [G,2E2] | "G#m" [G,2B2] [D,2B2] |
"Am" [A,2C2] [E,2C2] | "Bm" [B,2D2] [F,2D2] | "C#m" [C,2E2] [G,2E2] | "G#m" [G,2B2] [D,2B2] |

V:3
K:Amin
"A" z4 [c2e2] | "B" z4 [d2f2] | "C#" z4 [e2g2] | "G#" z4 [f2a2] |
"A" z4 [c2e2] | "B" z4 [d2f2] | "C#" z4 [e2g2] | "G#" z4 [f2a2] |

V:4
K:Amin
"A" [A,2E2] z4 | "B" [B,2F2] z4 | "C#" [c2G2] z4 | "G#" [g2B2] z4 |
"A" [A,2E2] z4 | "B" [B,2F2] z4 | "C#" [c2G2] z4 | "G#" [g2B2] z4 |"""

# Attempt to convert the ABC notation to MIDI using music21
try:
    from music21 import converter, midi

    # Convert ABC to music21 stream
    abc_work_final = converter.parse(abc_code_final, format='abc')

    # Create a MIDI file from the music21 stream
    midi_fp_final = 'cosmic_bloom_intro_final.mid'
    mf_final = midi.translate.music21ObjectToMidiFile(abc_work_final)
    mf_final.open(midi_fp_final, 'wb')
    mf_final.write()
    mf_final.close()

    midi_fp_final
    print("done.")

except Exception as e:
    print(str(e))

cannot process repeats on Stream that does not contain measures


In [43]:
from music21 import stream, note, tempo, metadata, instrument, dynamics

# Create a new score
score = stream.Score()

# Add metadata
score.metadata = metadata.Metadata()
score.metadata.title = "Cosmic Bloom"
score.metadata.composer = "Inspired by Hans Zimmer & John Williams"

# Set the tempo
score.append(tempo.MetronomeMark(number=91))

# Define instruments
instruments = [
    instrument.Violin(),
    instrument.Viola(),
    instrument.Violoncello(),
    instrument.Contrabass(),
    instrument.Flute(),
    instrument.Oboe(),
    instrument.Clarinet(),
    instrument.Bassoon(),
    instrument.FrenchHorn(),
    instrument.Trumpet(),
    instrument.Trombone(),
    instrument.Tuba(),
    instrument.Timpani(),
    instrument.Cymbals(),
    instrument.Harp()
]

# Define the main theme melody
melody_notes = [
    note.Rest(quarterLength=2),
    note.Note('A4', quarterLength=1),
    note.Note('B4', quarterLength=1),
    note.Rest(quarterLength=1),
    note.Note('C#5', quarterLength=1),
    note.Rest(quarterLength=1),
    note.Note('G#4', quarterLength=2),
    note.Rest(quarterLength=1),
    note.Note('A4', quarterLength=1),
    note.Rest(quarterLength=1),
    note.Note('B4', quarterLength=1),
    note.Rest(quarterLength=1),
    note.Note('C#5', quarterLength=1),
    note.Rest(quarterLength=1),
    note.Note('G#4', quarterLength=2),
]

# Create parts for each instrument and add the melody
for instr in instruments:
    part = stream.Part()
    part.append(instr)
    part.append(dynamics.Dynamic('mp'))  # Start with mezzo-piano
    part.append(melody_notes)
    score.append(part)

# Export to MusicXML
musicxml_path = 'cosmic_bloom.xml'
score.write('musicxml', fp=musicxml_path)

AttributeError: module 'music21.instrument' has no attribute 'FrenchHorn'